<a href="https://colab.research.google.com/github/benmsanderson/tcre-star/blob/main/ESGF_read_land_fluxes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.simplefilter("ignore")

from itertools import groupby

In [2]:
pip install nc-time-axis


In [3]:
pip install h5pyd

In [4]:
 pip install netcdf4==1.6.4

In [5]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import h5pyd
import xarray as xr
import nc_time_axis

xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Search using ESGF API

In [6]:
#!/usr/bin/env python
from __future__ import print_function
import requests
import xml.etree.ElementTree as ET
import numpy

# Author: Unknown
# I got the original version from a word document published by ESGF
# https://docs.google.com/document/d/1pxz1Kd3JHfFp8vR2JCVBfApbsHmbUQQstifhGNdc6U0/edit?usp=sharing

# API AT: https://github.com/ESGF/esgf.github.io/wiki/ESGF_Search_REST_API#results-pagination

def esgf_search(server="https://esgf-node.llnl.gov/esg-search/search",
                files_type="OPENDAP", local_node=True, project="CMIP6",
                verbose=False, format="application%2Fsolr%2Bjson",
                use_csrf=False, **search):
    client = requests.session()
    payload = search
    payload["project"] = project
    payload["type"]= "File"
    payload["latest"]="true"

    if local_node:
        payload["distrib"] = "false"
    if use_csrf:
        client.get(server)
        if 'csrftoken' in client.cookies:
            # Django 1.6 and up
            csrftoken = client.cookies['csrftoken']
        else:
            # older versions
            csrftoken = client.cookies['csrf']
        payload["csrfmiddlewaretoken"] = csrftoken

    payload["format"] = format

    offset = 0
    numFound = 10000
    all_files = []
    files_type = files_type.upper()
    while offset < numFound:
        payload["offset"] = offset
        url_keys = []
        for k in payload:
            url_keys += ["{}={}".format(k, payload[k])]

        url = "{}/?{}".format(server, "&".join(url_keys))
        #print(url)
        r = client.get(url)
        r.raise_for_status()
        resp = r.json()["response"]
        numFound = int(resp["numFound"])
        resp = resp["docs"]
        offset += len(resp)
        for d in resp:
            if verbose:
                for k in d:
                    print("{}: {}".format(k,d[k]))
            url = d["url"]
            for f in d["url"]:
                sp = f.split("|")
                if sp[-1] == files_type:
                    all_files.append(sp[0].split(".html")[0])
    result=sorted(all_files)
    tmp = [[c for _, c in g] for _, g in groupby(zip( [x.split('_')[-3] for x in result],result), key=lambda x: x[0])]
    return tmp

In [7]:
mdls=["CanESM5","GFDL-ESM4","GISS-E2-1-G","GISS-E2-1-H","CESM2-WACCM","MIROC-ES2L","CESM2-WACCM","UKESM1-0-LL","IPSL-CM6A-LR"]
runs=["r1i1p1f1","r1i1p1f1","r1i1p1f1","r1i1p1f1","r1i1p1f1","r1i1p1f1","r1i1p1f1","r1i1p1f1","r1i1p1f1"]

In [8]:
##esgf_search(activity_id='CMIP', table_id='Lmon',
  #                    experiment_id='historical', variable_id='nbp')

In [9]:
dset_34={}
dset_85={}

In [10]:
scens={'ssp119','ssp126','ssp245','ssp585','ssp534-over'}

In [11]:
var='nbp'
result_hist={}
result_scn={}

for i,mdl in enumerate(mdls):
  result_scn[mdl]={}
  result_hist[mdl] = esgf_search(activity_id='CMIP', table_id='Lmon',
                      experiment_id='historical', variable_id=var,
                      source_id=mdl)
  for i,scn in enumerate(scens):
    result_scn[mdl][scn] = esgf_search(activity_id='ScenarioMIP', table_id='Lmon',
                      experiment_id=scn, variable_id=var,
                      source_id=mdl)



In [12]:
result_area={}
for i,mdl in enumerate(mdls):

  result_area[mdl] = esgf_search(variable_id='areacella', activity_id='CMIP',
                          source_id=mdl)

In [13]:
mdls[:1]

['CanESM5']

In [14]:
ds_hist={}
for i,mdl in enumerate(mdls[:1]):
       if len(result_hist[mdl])>0:
         ds_hist[mdl] = xr.open_mfdataset(result_hist[mdl][0], combine='by_coords',use_cftime=True)


In [26]:
ds_scn={}
for i,mdl in enumerate(mdls[:1]):
    ds_scn[mdl]={}
    for i,scn in enumerate(scens):
       if len(result_scn[mdl][scn])>0:
          if scn=='ssp534-over':
              tmp0 = xr.open_mfdataset(result_scn[mdl]['ssp585'][0], combine='by_coords',use_cftime=True)
              tmp1 = xr.open_mfdataset(result_scn[mdl]['ssp534-over'][0], combine='by_coords',use_cftime=True)
              tmp=xr.concat((tmp0.sel(time=slice("1850-01-01", "2039-12-30")),tmp1.sel(time=slice("2040-01-01", "2300-12-30"))),'time')
          else:
              tmp = xr.open_mfdataset(result_scn[mdl][scn][0], combine='by_coords',use_cftime=True)
          ds_scn[mdl][scn]=xr.concat((ds_hist[mdl],tmp),'time')

In [25]:
ds_scn['CanESM5']['ssp534-over']

<xarray.Dataset>
Dimensions:    (time: 3012, bnds: 2, lat: 64, lon: 128)
Coordinates:
  * time       (time) object 1850-01-16 12:00:00 ... 2100-12-16 12:00:00
  * lat        (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
  * lon        (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds, time) float64 dask.array<chunksize=(64, 2, 1980), meta=np.ndarray>
    lon_bnds   (lon, bnds, time) float64 dask.array<chunksize=(128, 2, 1980), meta=np.ndarray>
    nbp        (time, lat, lon) float32 dask.array<chunksize=(1980, 64, 128), meta=np.ndarray>
Attributes: (12/54)
    CCCma_model_hash:                55f484f90aff0e32c5a8e92a42c6b9ae7ffe6224
    CCCma_parent_runid:              rc3.1-pictrl
    CCCma_pycmor_hash:               33c30511acc319a98240633965a04ca99c26427e
    CCCma_runid:                     rc3.1-his10
    Conventions:                     CF-1.7 CMIP-6.2
    YMDH_branch_time_in_child:       1850:01:01:00
    ...                              ...
    variable_id:                     nbp
    variant_label:                   r10i1p1f1
    version:                         v20190429
    license:                         CMIP6 model data produced by The Governm...
    cmor_version:                    3.4.0
    DODS_EXTRA.Unlimited_Dimension:  time

In [ ]:



        ds_534_end = xr.open_mfdataset(result_ssp534[0], combine='by_coords',use_cftime=True)
      ds_534=xr.concat((ds_585.sel(time=slice("1850-01-01", "2039-12-30")),ds_534_end),'time')

  files_area = esgf_search(variable_id='areacella', activity_id='CMIP',
                          source_id=mdl)
  ds_area = xr.open_dataset(files_area[0][0])

  total_area = ds_area.areacella.sum(dim=['lon', 'lat'])
  ts_585 = (ds_585[var] * ds_area.areacella.values).sum(dim=['lon', 'lat'])
  ts_534 = (ds_534[var] * ds_area.areacella.values).sum(dim=['lon', 'lat'])

  ts_534.load()
  ts_585.load()
  dset_34[mdl]=ts_534.copy()
  dset_85[mdl]=ts_585.copy()

IndexError: ignored

In [ ]:
esgf_search(activity_id='CMIP', table_id='Lmon',
                      experiment_id='historical', variable_id=var,
                      source_id=mdl)

In [ ]:
esgf_search(activity_id='ScenarioMIP', table_id='Lmon',
                      experiment_id='ssp585', variable_id=var,
                      source_id=mdl)